In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import time

import tensorflow as tf

import model_cifar10 as cifar10

In [2]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('train_dir', '/tmp/cifar10_train',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 5000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")
tf.app.flags.DEFINE_integer('log_frequency', 10,
                            """How often to log results to the console.""")



In [3]:
def train():
  """Train CIFAR-10 for a number of steps."""
  with tf.Graph().as_default():
    global_step = tf.contrib.framework.get_or_create_global_step()

    # Get images and labels for CIFAR-10.
    images, labels = cifar10.distorted_inputs()

    # Build a Graph that computes the logits predictions from the
    # inference model.
    logits = cifar10.inference(images)

    # Calculate loss.
    loss = cifar10.loss(logits, labels)

    # Build a Graph that trains the model with one batch of examples and
    # updates the model parameters.
    train_op = cifar10.train(loss, global_step)

    class _LoggerHook(tf.train.SessionRunHook):
      """Logs loss and runtime."""

      def begin(self):
        self._step = -1
        self._start_time = time.time()

      def before_run(self, run_context):
        self._step += 1
        return tf.train.SessionRunArgs(loss)  # Asks for loss value.

      def after_run(self, run_context, run_values):
        if self._step % FLAGS.log_frequency == 0:
          current_time = time.time()
          duration = current_time - self._start_time
          self._start_time = current_time

          loss_value = run_values.results
          examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
          sec_per_batch = float(duration / FLAGS.log_frequency)

          format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                        'sec/batch)')
          print (format_str % (datetime.now(), self._step, loss_value,
                               examples_per_sec, sec_per_batch))

    with tf.train.MonitoredTrainingSession(
        checkpoint_dir=FLAGS.train_dir,
        hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
               tf.train.NanTensorHook(loss),
               _LoggerHook()],
        config=tf.ConfigProto(
            log_device_placement=FLAGS.log_device_placement)) as mon_sess:
      while not mon_sess.should_stop():
        mon_sess.run(train_op)


In [ ]:
def main(argv=None):  # pylint: disable=unused-argument
  cifar10.maybe_download_and_extract()
  if tf.gfile.Exists(FLAGS.train_dir):
    tf.gfile.DeleteRecursively(FLAGS.train_dir)
  tf.gfile.MakeDirs(FLAGS.train_dir)
  train()


if __name__ == '__main__':
  tf.app.run()

In [ ]:
'''
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
INFO:tensorflow:Summary name conv1/weight_loss (raw) is illegal; using conv1/weight_loss__raw_ instead.
INFO:tensorflow:Summary name conv2/weight_loss (raw) is illegal; using conv2/weight_loss__raw_ instead.
INFO:tensorflow:Summary name local3/weight_loss (raw) is illegal; using local3/weight_loss__raw_ instead.
INFO:tensorflow:Summary name local4/weight_loss (raw) is illegal; using local4/weight_loss__raw_ instead.
INFO:tensorflow:Summary name softmax_linear/weight_loss (raw) is illegal; using softmax_linear/weight_loss__raw_ instead.
INFO:tensorflow:Summary name cross_entropy (raw) is illegal; using cross_entropy__raw_ instead.
INFO:tensorflow:Summary name total_loss (raw) is illegal; using total_loss__raw_ instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/cifar10_train/model.ckpt.
2017-04-02 00:16:19.959166: step 0, loss = 4.67 (227.1 examples/sec; 0.564 sec/batch)
2017-04-02 00:16:28.469408: step 10, loss = 4.62 (150.4 examples/sec; 0.851 sec/batch)
2017-04-02 00:16:36.858377: step 20, loss = 4.43 (152.6 examples/sec; 0.839 sec/batch)
2017-04-02 00:16:45.239960: step 30, loss = 4.45 (152.7 examples/sec; 0.838 sec/batch)
2017-04-02 00:16:53.633181: step 40, loss = 4.41 (152.5 examples/sec; 0.839 sec/batch)
2017-04-02 00:17:01.983407: step 50, loss = 4.24 (153.3 examples/sec; 0.835 sec/batch)
2017-04-02 00:17:10.348189: step 60, loss = 4.27 (153.0 examples/sec; 0.836 sec/batch)
2017-04-02 00:17:18.695242: step 70, loss = 4.36 (153.3 examples/sec; 0.835 sec/batch)
2017-04-02 00:17:27.062691: step 80, loss = 4.13 (153.0 examples/sec; 0.837 sec/batch)
2017-04-02 00:17:35.396549: step 90, loss = 4.11 (153.6 examples/sec; 0.833 sec/batch)
INFO:tensorflow:global_step/sec: 1.18773
2017-04-02 00:17:44.040240: step 100, loss = 4.14 (148.1 examples/sec; 0.864 sec/batch)
2017-04-02 00:17:52.378130: step 110, loss = 4.34 (153.5 examples/sec; 0.834 sec/batch)
2017-04-02 00:18:00.723685: step 120, loss = 4.06 (153.4 examples/sec; 0.835 sec/batch)
2017-04-02 00:18:09.077576: step 130, loss = 4.05 (153.2 examples/sec; 0.835 sec/batch)
2017-04-02 00:18:17.447887: step 140, loss = 3.85 (152.9 examples/sec; 0.837 sec/batch)
2017-04-02 00:18:26.882781: step 150, loss = 4.05 (135.7 examples/sec; 0.943 sec/batch)
2017-04-02 00:18:36.063995: step 160, loss = 4.08 (139.4 examples/sec; 0.918 sec/batch)
2017-04-02 00:18:45.930570: step 170, loss = 3.87 (129.7 examples/sec; 0.987 sec/batch)
2017-04-02 00:18:54.764675: step 180, loss = 3.83 (144.9 examples/sec; 0.883 sec/batch)
2017-04-02 00:19:03.712052: step 190, loss = 3.90 (143.1 examples/sec; 0.895 sec/batch)
INFO:tensorflow:global_step/sec: 1.12588
2017-04-02 00:19:12.859296: step 200, loss = 3.68 (139.9 examples/sec; 0.915 sec/batch)
2017-04-02 00:19:21.716205: step 210, loss = 3.76 (144.5 examples/sec; 0.886 sec/batch)
2017-04-02 00:19:30.052570: step 220, loss = 3.65 (153.5 examples/sec; 0.834 sec/batch)
2017-04-02 00:19:38.377081: step 230, loss = 3.86 (153.8 examples/sec; 0.832 sec/batch)
2017-04-02 00:19:46.689511: step 240, loss = 3.72 (154.0 examples/sec; 0.831 sec/batch)
2017-04-02 00:19:55.029124: step 250, loss = 3.59 (153.5 examples/sec; 0.834 sec/batch)
2017-04-02 00:20:03.390222: step 260, loss = 3.69 (153.1 examples/sec; 0.836 sec/batch)
2017-04-02 00:20:11.702152: step 270, loss = 3.70 (154.0 examples/sec; 0.831 sec/batch)
2017-04-02 00:20:20.025726: step 280, loss = 3.66 (153.8 examples/sec; 0.832 sec/batch)
2017-04-02 00:20:28.373714: step 290, loss = 3.73 (153.3 examples/sec; 0.835 sec/batch)
INFO:tensorflow:global_step/sec: 1.18974
2017-04-02 00:20:36.911680: step 300, loss = 3.62 (149.9 examples/sec; 0.854 sec/batch)
2017-04-02 00:20:45.248946: step 310, loss = 3.59 (153.5 examples/sec; 0.834 sec/batch)
2017-04-02 00:20:53.541087: step 320, loss = 3.53 (154.4 examples/sec; 0.829 sec/batch)
2017-04-02 00:21:02.559480: step 330, loss = 3.46 (141.9 examples/sec; 0.902 sec/batch)
2017-04-02 00:21:11.797169: step 340, loss = 3.33 (138.6 examples/sec; 0.924 sec/batch)
2017-04-02 00:21:21.047720: step 350, loss = 3.38 (138.4 examples/sec; 0.925 sec/batch)
2017-04-02 00:21:30.329156: step 360, loss = 3.43 (137.9 examples/sec; 0.928 sec/batch)
2017-04-02 00:21:39.362127: step 370, loss = 3.34 (141.7 examples/sec; 0.903 sec/batch)
2017-04-02 00:21:48.342273: step 380, loss = 3.39 (142.5 examples/sec; 0.898 sec/batch)
2017-04-02 00:21:57.088372: step 390, loss = 3.60 (146.4 examples/sec; 0.875 sec/batch)
INFO:tensorflow:global_step/sec: 1.11115
2017-04-02 00:22:06.908824: step 400, loss = 3.28 (130.3 examples/sec; 0.982 sec/batch)
2017-04-02 00:22:15.397568: step 410, loss = 3.39 (150.8 examples/sec; 0.849 sec/batch)
2017-04-02 00:22:23.788091: step 420, loss = 3.18 (152.6 examples/sec; 0.839 sec/batch)
2017-04-02 00:22:32.122700: step 430, loss = 3.14 (153.6 examples/sec; 0.833 sec/batch)
2017-04-02 00:22:40.406278: step 440, loss = 3.23 (154.5 examples/sec; 0.828 sec/batch)
2017-04-02 00:22:48.718129: step 450, loss = 3.32 (154.0 examples/sec; 0.831 sec/batch)
2017-04-02 00:22:56.990501: step 460, loss = 3.17 (154.7 examples/sec; 0.827 sec/batch)
2017-04-02 00:23:06.700121: step 470, loss = 3.05 (131.8 examples/sec; 0.971 sec/batch)
2017-04-02 00:23:15.580084: step 480, loss = 3.10 (144.1 examples/sec; 0.888 sec/batch)
2017-04-02 00:23:25.991335: step 490, loss = 3.01 (122.9 examples/sec; 1.041 sec/batch)
INFO:tensorflow:global_step/sec: 1.11582
2017-04-02 00:23:36.529065: step 500, loss = 3.19 (121.5 examples/sec; 1.054 sec/batch)
2017-04-02 00:23:44.975973: step 510, loss = 3.11 (151.5 examples/sec; 0.845 sec/batch)
2017-04-02 00:23:53.316954: step 520, loss = 3.08 (153.5 examples/sec; 0.834 sec/batch)
2017-04-02 00:24:01.866992: step 530, loss = 3.09 (149.7 examples/sec; 0.855 sec/batch)
2017-04-02 00:24:10.487928: step 540, loss = 3.20 (148.5 examples/sec; 0.862 sec/batch)
2017-04-02 00:24:18.962554: step 550, loss = 3.06 (151.0 examples/sec; 0.847 sec/batch)
2017-04-02 00:24:27.668545: step 560, loss = 3.16 (147.0 examples/sec; 0.871 sec/batch)
2017-04-02 00:24:36.078681: step 570, loss = 2.86 (152.2 examples/sec; 0.841 sec/batch)
2017-04-02 00:24:44.772606: step 580, loss = 2.90 (147.2 examples/sec; 0.869 sec/batch)
2017-04-02 00:24:53.802519: step 590, loss = 2.85 (141.8 examples/sec; 0.903 sec/batch)
INFO:tensorflow:global_step/sec: 1.15483
2017-04-02 00:25:03.122261: step 600, loss = 3.10 (137.3 examples/sec; 0.932 sec/batch)
2017-04-02 00:25:12.254943: step 610, loss = 3.02 (140.2 examples/sec; 0.913 sec/batch)
2017-04-02 00:25:20.808049: step 620, loss = 3.19 (149.7 examples/sec; 0.855 sec/batch)
2017-04-02 00:25:30.170681: step 630, loss = 2.99 (136.7 examples/sec; 0.936 sec/batch)
2017-04-02 00:25:39.007952: step 640, loss = 2.94 (144.8 examples/sec; 0.884 sec/batch)
2017-04-02 00:25:47.871718: step 650, loss = 3.05 (144.4 examples/sec; 0.886 sec/batch)
2017-04-02 00:25:56.403477: step 660, loss = 2.87 (150.0 examples/sec; 0.853 sec/batch)
2017-04-02 00:26:05.643995: step 670, loss = 2.82 (138.5 examples/sec; 0.924 sec/batch)
2017-04-02 00:26:14.607179: step 680, loss = 2.96 (142.8 examples/sec; 0.896 sec/batch)
INFO:tensorflow:Saving checkpoints for 686 into /tmp/cifar10_train/model.ckpt.
2017-04-02 00:26:25.476270: step 690, loss = 2.78 (117.8 examples/sec; 1.087 sec/batch)
INFO:tensorflow:global_step/sec: 1.07853
2017-04-02 00:26:35.841064: step 700, loss = 3.13 (123.5 examples/sec; 1.036 sec/batch)
2017-04-02 00:26:44.684927: step 710, loss = 2.89 (144.7 examples/sec; 0.884 sec/batch)
2017-04-02 00:26:55.146595: step 720, loss = 2.86 (122.4 examples/sec; 1.046 sec/batch)
2017-04-02 00:27:04.935700: step 730, loss = 2.82 (130.8 examples/sec; 0.979 sec/batch)
2017-04-02 00:27:14.285600: step 740, loss = 2.67 (136.9 examples/sec; 0.935 sec/batch)
2017-04-02 00:27:25.534495: step 750, loss = 2.80 (113.8 examples/sec; 1.125 sec/batch)
2017-04-02 00:27:36.558184: step 760, loss = 2.64 (116.1 examples/sec; 1.102 sec/batch)
2017-04-02 00:27:47.781091: step 770, loss = 2.95 (114.1 examples/sec; 1.122 sec/batch)
2017-04-02 00:27:57.279515: step 780, loss = 2.70 (134.8 examples/sec; 0.950 sec/batch)
2017-04-02 00:28:09.166274: step 790, loss = 2.81 (107.7 examples/sec; 1.189 sec/batch)
INFO:tensorflow:global_step/sec: 0.975615
2017-04-02 00:28:18.340621: step 800, loss = 2.66 (139.5 examples/sec; 0.917 sec/batch)
2017-04-02 00:28:27.788816: step 810, loss = 2.91 (135.5 examples/sec; 0.945 sec/batch)
2017-04-02 00:28:37.947026: step 820, loss = 2.51 (126.0 examples/sec; 1.016 sec/batch)
2017-04-02 00:28:48.601610: step 830, loss = 2.72 (120.1 examples/sec; 1.065 sec/batch)
2017-04-02 00:28:56.926551: step 840, loss = 2.85 (153.8 examples/sec; 0.832 sec/batch)
2017-04-02 00:29:07.479655: step 850, loss = 2.64 (121.3 examples/sec; 1.055 sec/batch)
2017-04-02 00:29:17.917144: step 860, loss = 2.66 (122.6 examples/sec; 1.044 sec/batch)
2017-04-02 00:29:26.180198: step 870, loss = 2.69 (154.9 examples/sec; 0.826 sec/batch)
2017-04-02 00:29:34.824031: step 880, loss = 2.53 (148.1 examples/sec; 0.864 sec/batch)
2017-04-02 00:29:43.202427: step 890, loss = 2.66 (152.8 examples/sec; 0.838 sec/batch)
INFO:tensorflow:global_step/sec: 1.07232
2017-04-02 00:29:51.595881: step 900, loss = 2.42 (152.5 examples/sec; 0.839 sec/batch)
2017-04-02 00:29:59.870267: step 910, loss = 2.65 (154.7 examples/sec; 0.827 sec/batch)
2017-04-02 00:30:08.164065: step 920, loss = 2.59 (154.3 examples/sec; 0.829 sec/batch)
2017-04-02 00:30:16.415181: step 930, loss = 2.40 (155.1 examples/sec; 0.825 sec/batch)
2017-04-02 00:30:24.742014: step 940, loss = 2.53 (153.7 examples/sec; 0.833 sec/batch)
2017-04-02 00:30:32.976296: step 950, loss = 2.44 (155.4 examples/sec; 0.823 sec/batch)
2017-04-02 00:30:43.383843: step 960, loss = 2.65 (123.0 examples/sec; 1.041 sec/batch)
2017-04-02 00:30:51.647636: step 970, loss = 2.59 (154.9 examples/sec; 0.826 sec/batch)
2017-04-02 00:31:00.018611: step 980, loss = 2.75 (152.9 examples/sec; 0.837 sec/batch)
2017-04-02 00:31:10.737087: step 990, loss = 2.38 (119.4 examples/sec; 1.072 sec/batch)
INFO:tensorflow:global_step/sec: 1.14182
2017-04-02 00:31:19.175098: step 1000, loss = 2.30 (151.7 examples/sec; 0.844 sec/batch)
2017-04-02 00:31:27.449730: step 1010, loss = 2.41 (154.7 examples/sec; 0.827 sec/batch)
2017-04-02 00:31:35.699527: step 1020, loss = 2.37 (155.2 examples/sec; 0.825 sec/batch)
2017-04-02 00:31:43.939009: step 1030, loss = 2.94 (155.3 examples/sec; 0.824 sec/batch)
2017-04-02 00:31:54.322276: step 1040, loss = 2.47 (123.3 examples/sec; 1.038 sec/batch)
2017-04-02 00:32:02.536808: step 1050, loss = 2.31 (155.8 examples/sec; 0.821 sec/batch)
2017-04-02 00:32:10.807647: step 1060, loss = 2.49 (154.8 examples/sec; 0.827 sec/batch)
2017-04-02 00:32:19.028184: step 1070, loss = 2.51 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 00:32:27.330531: step 1080, loss = 2.42 (154.2 examples/sec; 0.830 sec/batch)
2017-04-02 00:32:35.581442: step 1090, loss = 2.39 (155.1 examples/sec; 0.825 sec/batch)
INFO:tensorflow:global_step/sec: 1.17932
2017-04-02 00:32:43.969371: step 1100, loss = 2.19 (152.6 examples/sec; 0.839 sec/batch)
2017-04-02 00:32:52.257971: step 1110, loss = 2.53 (154.4 examples/sec; 0.829 sec/batch)
2017-04-02 00:33:00.514316: step 1120, loss = 2.31 (155.0 examples/sec; 0.826 sec/batch)
2017-04-02 00:33:10.904530: step 1130, loss = 2.44 (123.2 examples/sec; 1.039 sec/batch)
2017-04-02 00:33:19.166583: step 1140, loss = 2.31 (154.9 examples/sec; 0.826 sec/batch)
2017-04-02 00:33:27.416705: step 1150, loss = 2.30 (155.1 examples/sec; 0.825 sec/batch)
2017-04-02 00:33:35.970321: step 1160, loss = 1.93 (149.6 examples/sec; 0.855 sec/batch)
2017-04-02 00:33:44.223339: step 1170, loss = 2.15 (155.1 examples/sec; 0.825 sec/batch)
2017-04-02 00:33:54.660414: step 1180, loss = 2.06 (122.6 examples/sec; 1.044 sec/batch)
2017-04-02 00:34:02.935331: step 1190, loss = 2.45 (154.7 examples/sec; 0.827 sec/batch)
INFO:tensorflow:global_step/sec: 1.14503
2017-04-02 00:34:11.303518: step 1200, loss = 2.37 (153.0 examples/sec; 0.837 sec/batch)
2017-04-02 00:34:21.792043: step 1210, loss = 2.24 (122.0 examples/sec; 1.049 sec/batch)
2017-04-02 00:34:30.069063: step 1220, loss = 2.19 (154.6 examples/sec; 0.828 sec/batch)
2017-04-02 00:34:38.296274: step 1230, loss = 2.20 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 00:34:46.522901: step 1240, loss = 2.16 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 00:34:54.735389: step 1250, loss = 2.17 (155.9 examples/sec; 0.821 sec/batch)
2017-04-02 00:35:02.980234: step 1260, loss = 2.29 (155.2 examples/sec; 0.824 sec/batch)
2017-04-02 00:35:11.225870: step 1270, loss = 2.20 (155.2 examples/sec; 0.825 sec/batch)
2017-04-02 00:35:19.473088: step 1280, loss = 2.17 (155.2 examples/sec; 0.825 sec/batch)
2017-04-02 00:35:27.722140: step 1290, loss = 2.23 (155.2 examples/sec; 0.825 sec/batch)
INFO:tensorflow:global_step/sec: 1.17935
2017-04-02 00:35:36.095863: step 1300, loss = 2.23 (152.9 examples/sec; 0.837 sec/batch)
2017-04-02 00:35:44.358779: step 1310, loss = 2.16 (154.9 examples/sec; 0.826 sec/batch)
2017-04-02 00:35:54.747354: step 1320, loss = 2.07 (123.2 examples/sec; 1.039 sec/batch)
2017-04-02 00:36:02.985290: step 1330, loss = 2.11 (155.4 examples/sec; 0.824 sec/batch)
2017-04-02 00:36:11.218659: step 1340, loss = 2.17 (155.5 examples/sec; 0.823 sec/batch)
2017-04-02 00:36:19.438175: step 1350, loss = 2.03 (155.7 examples/sec; 0.822 sec/batch)
INFO:tensorflow:Saving checkpoints for 1353 into /tmp/cifar10_train/model.ckpt.
2017-04-02 00:36:27.854029: step 1360, loss = 2.23 (152.1 examples/sec; 0.842 sec/batch)
2017-04-02 00:36:36.082129: step 1370, loss = 2.26 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 00:36:46.511959: step 1380, loss = 2.13 (122.7 examples/sec; 1.043 sec/batch)
2017-04-02 00:36:54.762485: step 1390, loss = 2.16 (155.1 examples/sec; 0.825 sec/batch)
INFO:tensorflow:global_step/sec: 1.14932
2017-04-02 00:37:03.103651: step 1400, loss = 2.18 (153.5 examples/sec; 0.834 sec/batch)
2017-04-02 00:37:11.338882: step 1410, loss = 1.96 (155.4 examples/sec; 0.824 sec/batch)
2017-04-02 00:37:19.561831: step 1420, loss = 2.01 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 00:37:29.957076: step 1430, loss = 2.15 (123.1 examples/sec; 1.040 sec/batch)
2017-04-02 00:37:40.319316: step 1440, loss = 1.88 (123.5 examples/sec; 1.036 sec/batch)
2017-04-02 00:37:48.524043: step 1450, loss = 1.96 (156.0 examples/sec; 0.820 sec/batch)
2017-04-02 00:37:56.789435: step 1460, loss = 1.98 (154.9 examples/sec; 0.827 sec/batch)
2017-04-02 00:38:05.023175: step 1470, loss = 1.95 (155.5 examples/sec; 0.823 sec/batch)
2017-04-02 00:38:13.260107: step 1480, loss = 2.21 (155.4 examples/sec; 0.824 sec/batch)
2017-04-02 00:38:21.528371: step 1490, loss = 2.02 (154.8 examples/sec; 0.827 sec/batch)
INFO:tensorflow:global_step/sec: 1.15213
2017-04-02 00:38:29.899819: step 1500, loss = 1.99 (152.9 examples/sec; 0.837 sec/batch)
2017-04-02 00:38:38.484575: step 1510, loss = 2.11 (149.1 examples/sec; 0.858 sec/batch)
2017-04-02 00:38:46.714053: step 1520, loss = 1.93 (155.5 examples/sec; 0.823 sec/batch)
2017-04-02 00:38:57.122591: step 1530, loss = 2.07 (123.0 examples/sec; 1.041 sec/batch)
2017-04-02 00:39:05.369738: step 1540, loss = 2.03 (155.2 examples/sec; 0.825 sec/batch)
2017-04-02 00:39:15.760091: step 1550, loss = 1.90 (123.2 examples/sec; 1.039 sec/batch)
2017-04-02 00:39:23.997204: step 1560, loss = 1.93 (155.4 examples/sec; 0.824 sec/batch)
2017-04-02 00:39:32.202753: step 1570, loss = 1.88 (156.0 examples/sec; 0.821 sec/batch)
2017-04-02 00:39:40.428202: step 1580, loss = 1.83 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 00:39:50.795915: step 1590, loss = 1.71 (123.5 examples/sec; 1.037 sec/batch)
INFO:tensorflow:global_step/sec: 1.12039
2017-04-02 00:39:59.154506: step 1600, loss = 1.80 (153.1 examples/sec; 0.836 sec/batch)
2017-04-02 00:40:07.430418: step 1610, loss = 1.92 (154.7 examples/sec; 0.828 sec/batch)
2017-04-02 00:40:15.656729: step 1620, loss = 1.76 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 00:40:23.940375: step 1630, loss = 1.88 (154.5 examples/sec; 0.828 sec/batch)
2017-04-02 00:40:32.165560: step 1640, loss = 1.87 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 00:40:40.413540: step 1650, loss = 1.85 (155.2 examples/sec; 0.825 sec/batch)
2017-04-02 00:40:50.785676: step 1660, loss = 1.79 (123.4 examples/sec; 1.037 sec/batch)
2017-04-02 00:40:59.012683: step 1670, loss = 1.73 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 00:41:09.398013: step 1680, loss = 1.96 (123.3 examples/sec; 1.039 sec/batch)
2017-04-02 00:41:17.636340: step 1690, loss = 1.81 (155.4 examples/sec; 0.824 sec/batch)
INFO:tensorflow:global_step/sec: 1.15196
2017-04-02 00:41:25.963299: step 1700, loss = 1.91 (153.7 examples/sec; 0.833 sec/batch)
2017-04-02 00:41:34.232392: step 1710, loss = 1.95 (154.8 examples/sec; 0.827 sec/batch)
2017-04-02 00:41:44.621376: step 1720, loss = 1.65 (123.2 examples/sec; 1.039 sec/batch)
2017-04-02 00:41:54.974612: step 1730, loss = 1.65 (123.6 examples/sec; 1.035 sec/batch)
2017-04-02 00:42:05.341385: step 1740, loss = 1.85 (123.5 examples/sec; 1.037 sec/batch)
2017-04-02 00:42:13.570870: step 1750, loss = 1.58 (155.5 examples/sec; 0.823 sec/batch)
2017-04-02 00:42:21.843442: step 1760, loss = 2.08 (154.7 examples/sec; 0.827 sec/batch)
2017-04-02 00:42:30.429011: step 1770, loss = 1.78 (149.1 examples/sec; 0.859 sec/batch)
2017-04-02 00:42:40.377325: step 1780, loss = 1.81 (128.7 examples/sec; 0.995 sec/batch)
2017-04-02 00:42:48.570562: step 1790, loss = 1.83 (156.2 examples/sec; 0.819 sec/batch)
INFO:tensorflow:global_step/sec: 1.09926
2017-04-02 00:42:56.933884: step 1800, loss = 1.67 (153.0 examples/sec; 0.836 sec/batch)
2017-04-02 00:43:05.197635: step 1810, loss = 1.57 (154.9 examples/sec; 0.826 sec/batch)
2017-04-02 00:43:15.561999: step 1820, loss = 1.91 (123.5 examples/sec; 1.036 sec/batch)
2017-04-02 00:43:23.793051: step 1830, loss = 1.74 (155.5 examples/sec; 0.823 sec/batch)
2017-04-02 00:43:31.990505: step 1840, loss = 1.85 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 00:43:42.677958: step 1850, loss = 1.69 (119.8 examples/sec; 1.069 sec/batch)
2017-04-02 00:43:50.899693: step 1860, loss = 1.75 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 00:43:59.108227: step 1870, loss = 1.91 (155.9 examples/sec; 0.821 sec/batch)
2017-04-02 00:44:08.947155: step 1880, loss = 1.65 (130.1 examples/sec; 0.984 sec/batch)
2017-04-02 00:44:17.718409: step 1890, loss = 1.73 (145.9 examples/sec; 0.877 sec/batch)
INFO:tensorflow:global_step/sec: 1.1208
2017-04-02 00:44:26.155823: step 1900, loss = 1.54 (151.7 examples/sec; 0.844 sec/batch)
2017-04-02 00:44:34.401045: step 1910, loss = 1.52 (155.2 examples/sec; 0.825 sec/batch)
2017-04-02 00:44:43.514901: step 1920, loss = 1.60 (140.4 examples/sec; 0.911 sec/batch)
2017-04-02 00:44:55.122652: step 1930, loss = 1.70 (110.3 examples/sec; 1.161 sec/batch)
2017-04-02 00:45:03.329600: step 1940, loss = 1.56 (156.0 examples/sec; 0.821 sec/batch)
2017-04-02 00:45:11.534016: step 1950, loss = 1.66 (156.0 examples/sec; 0.820 sec/batch)
2017-04-02 00:45:19.736269: step 1960, loss = 1.74 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 00:45:27.955176: step 1970, loss = 1.51 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 00:45:38.303659: step 1980, loss = 1.64 (123.7 examples/sec; 1.035 sec/batch)
2017-04-02 00:45:48.684066: step 1990, loss = 1.54 (123.3 examples/sec; 1.038 sec/batch)
INFO:tensorflow:global_step/sec: 1.10055
2017-04-02 00:45:57.019691: step 2000, loss = 1.46 (153.6 examples/sec; 0.834 sec/batch)
2017-04-02 00:46:05.252465: step 2010, loss = 1.66 (155.5 examples/sec; 0.823 sec/batch)
2017-04-02 00:46:15.614958: step 2020, loss = 1.79 (123.5 examples/sec; 1.036 sec/batch)
INFO:tensorflow:Saving checkpoints for 2028 into /tmp/cifar10_train/model.ckpt.
2017-04-02 00:46:24.020134: step 2030, loss = 1.58 (152.3 examples/sec; 0.841 sec/batch)
2017-04-02 00:46:34.416687: step 2040, loss = 1.53 (123.1 examples/sec; 1.040 sec/batch)
2017-04-02 00:46:42.652165: step 2050, loss = 1.70 (155.4 examples/sec; 0.824 sec/batch)
2017-04-02 00:46:50.876775: step 2060, loss = 1.59 (155.6 examples/sec; 0.822 sec/batch)
2017-04-02 00:46:59.064611: step 2070, loss = 1.57 (156.3 examples/sec; 0.819 sec/batch)
2017-04-02 00:47:07.278703: step 2080, loss = 1.49 (155.8 examples/sec; 0.821 sec/batch)
2017-04-02 00:47:17.111032: step 2090, loss = 1.73 (130.2 examples/sec; 0.983 sec/batch)
INFO:tensorflow:global_step/sec: 1.09815
2017-04-02 00:47:28.082488: step 2100, loss = 1.58 (116.7 examples/sec; 1.097 sec/batch)
2017-04-02 00:47:36.355431: step 2110, loss = 1.52 (154.7 examples/sec; 0.827 sec/batch)
2017-04-02 00:47:44.581283: step 2120, loss = 1.52 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 00:47:52.803813: step 2130, loss = 1.67 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 00:48:01.030232: step 2140, loss = 1.83 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 00:48:11.400346: step 2150, loss = 1.44 (123.4 examples/sec; 1.037 sec/batch)
2017-04-02 00:48:19.593865: step 2160, loss = 1.59 (156.2 examples/sec; 0.819 sec/batch)
2017-04-02 00:48:27.889097: step 2170, loss = 1.53 (154.3 examples/sec; 0.830 sec/batch)
2017-04-02 00:48:38.581060: step 2180, loss = 1.45 (119.7 examples/sec; 1.069 sec/batch)
2017-04-02 00:48:47.669597: step 2190, loss = 1.59 (140.8 examples/sec; 0.909 sec/batch)
INFO:tensorflow:global_step/sec: 1.09298
2017-04-02 00:48:59.575564: step 2200, loss = 1.64 (107.5 examples/sec; 1.191 sec/batch)
2017-04-02 00:49:07.794601: step 2210, loss = 1.74 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 00:49:16.024041: step 2220, loss = 1.49 (155.5 examples/sec; 0.823 sec/batch)
2017-04-02 00:49:24.218654: step 2230, loss = 1.65 (156.2 examples/sec; 0.819 sec/batch)
2017-04-02 00:49:34.602998: step 2240, loss = 1.75 (123.3 examples/sec; 1.038 sec/batch)
2017-04-02 00:49:42.801572: step 2250, loss = 1.69 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 00:49:51.008949: step 2260, loss = 1.56 (156.0 examples/sec; 0.821 sec/batch)
2017-04-02 00:50:03.633130: step 2270, loss = 1.56 (101.4 examples/sec; 1.262 sec/batch)
2017-04-02 00:50:11.863158: step 2280, loss = 1.54 (155.5 examples/sec; 0.823 sec/batch)
2017-04-02 00:50:20.045894: step 2290, loss = 1.58 (156.4 examples/sec; 0.818 sec/batch)
INFO:tensorflow:global_step/sec: 1.09873
2017-04-02 00:50:30.590253: step 2300, loss = 1.48 (121.4 examples/sec; 1.054 sec/batch)
2017-04-02 00:50:38.789752: step 2310, loss = 1.40 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 00:50:51.438417: step 2320, loss = 1.50 (101.2 examples/sec; 1.265 sec/batch)
2017-04-02 00:50:59.623934: step 2330, loss = 1.50 (156.4 examples/sec; 0.819 sec/batch)
2017-04-02 00:51:07.848865: step 2340, loss = 1.26 (155.6 examples/sec; 0.822 sec/batch)
2017-04-02 00:51:16.048894: step 2350, loss = 1.41 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 00:51:24.268735: step 2360, loss = 1.55 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 00:51:32.466651: step 2370, loss = 1.74 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 00:51:43.480512: step 2380, loss = 1.44 (116.2 examples/sec; 1.101 sec/batch)
2017-04-02 00:51:53.200096: step 2390, loss = 1.76 (131.7 examples/sec; 0.972 sec/batch)
INFO:tensorflow:global_step/sec: 1.05333
2017-04-02 00:52:05.529536: step 2400, loss = 1.45 (103.8 examples/sec; 1.233 sec/batch)
2017-04-02 00:52:14.222726: step 2410, loss = 1.56 (147.2 examples/sec; 0.869 sec/batch)
2017-04-02 00:52:22.475851: step 2420, loss = 1.35 (155.1 examples/sec; 0.825 sec/batch)
2017-04-02 00:52:30.690199: step 2430, loss = 1.52 (155.8 examples/sec; 0.821 sec/batch)
2017-04-02 00:52:41.052945: step 2440, loss = 1.38 (123.5 examples/sec; 1.036 sec/batch)
2017-04-02 00:52:51.403141: step 2450, loss = 1.64 (123.7 examples/sec; 1.035 sec/batch)
2017-04-02 00:52:59.608487: step 2460, loss = 1.38 (156.0 examples/sec; 0.821 sec/batch)
2017-04-02 00:53:09.484047: step 2470, loss = 1.47 (129.6 examples/sec; 0.988 sec/batch)
2017-04-02 00:53:18.181559: step 2480, loss = 1.54 (147.2 examples/sec; 0.870 sec/batch)
2017-04-02 00:53:26.382919: step 2490, loss = 1.44 (156.1 examples/sec; 0.820 sec/batch)
INFO:tensorflow:global_step/sec: 1.11313
2017-04-02 00:53:35.367846: step 2500, loss = 1.29 (142.5 examples/sec; 0.898 sec/batch)
2017-04-02 00:53:45.321536: step 2510, loss = 1.44 (128.6 examples/sec; 0.995 sec/batch)
2017-04-02 00:53:57.406413: step 2520, loss = 1.53 (105.9 examples/sec; 1.208 sec/batch)
2017-04-02 00:54:06.185158: step 2530, loss = 1.37 (145.8 examples/sec; 0.878 sec/batch)
2017-04-02 00:54:14.392665: step 2540, loss = 1.35 (156.0 examples/sec; 0.821 sec/batch)
2017-04-02 00:54:22.657101: step 2550, loss = 1.47 (154.9 examples/sec; 0.826 sec/batch)
2017-04-02 00:54:30.893450: step 2560, loss = 1.49 (155.4 examples/sec; 0.824 sec/batch)
2017-04-02 00:54:39.782296: step 2570, loss = 1.38 (144.0 examples/sec; 0.889 sec/batch)
2017-04-02 00:54:51.614371: step 2580, loss = 1.33 (108.2 examples/sec; 1.183 sec/batch)
2017-04-02 00:54:59.822686: step 2590, loss = 1.39 (155.9 examples/sec; 0.821 sec/batch)
INFO:tensorflow:global_step/sec: 1.05506
2017-04-02 00:55:10.147854: step 2600, loss = 1.33 (124.0 examples/sec; 1.033 sec/batch)
2017-04-02 00:55:18.516701: step 2610, loss = 1.30 (152.9 examples/sec; 0.837 sec/batch)
2017-04-02 00:55:26.717557: step 2620, loss = 1.44 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 00:55:34.920804: step 2630, loss = 1.41 (156.0 examples/sec; 0.820 sec/batch)
2017-04-02 00:55:43.736736: step 2640, loss = 1.45 (145.2 examples/sec; 0.882 sec/batch)
2017-04-02 00:55:53.449297: step 2650, loss = 1.21 (131.8 examples/sec; 0.971 sec/batch)
2017-04-02 00:56:01.663532: step 2660, loss = 1.47 (155.8 examples/sec; 0.821 sec/batch)
2017-04-02 00:56:09.874669: step 2670, loss = 1.37 (155.9 examples/sec; 0.821 sec/batch)
2017-04-02 00:56:18.056053: step 2680, loss = 1.32 (156.5 examples/sec; 0.818 sec/batch)
INFO:tensorflow:Saving checkpoints for 2686 into /tmp/cifar10_train/model.ckpt.
2017-04-02 00:56:28.572287: step 2690, loss = 1.36 (121.7 examples/sec; 1.052 sec/batch)
INFO:tensorflow:global_step/sec: 1.15292
2017-04-02 00:56:36.881620: step 2700, loss = 1.40 (154.0 examples/sec; 0.831 sec/batch)
2017-04-02 00:56:45.153208: step 2710, loss = 1.23 (154.7 examples/sec; 0.827 sec/batch)
2017-04-02 00:56:55.534123: step 2720, loss = 1.33 (123.3 examples/sec; 1.038 sec/batch)
2017-04-02 00:57:03.735513: step 2730, loss = 1.20 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 00:57:11.945969: step 2740, loss = 1.24 (155.9 examples/sec; 0.821 sec/batch)
2017-04-02 00:57:22.286878: step 2750, loss = 1.40 (123.8 examples/sec; 1.034 sec/batch)
2017-04-02 00:57:32.656730: step 2760, loss = 1.26 (123.4 examples/sec; 1.037 sec/batch)
2017-04-02 00:57:40.828657: step 2770, loss = 1.37 (156.6 examples/sec; 0.817 sec/batch)
2017-04-02 00:57:51.199595: step 2780, loss = 1.39 (123.4 examples/sec; 1.037 sec/batch)
2017-04-02 00:57:59.394930: step 2790, loss = 1.30 (156.2 examples/sec; 0.820 sec/batch)
INFO:tensorflow:global_step/sec: 1.10059
2017-04-02 00:58:07.741949: step 2800, loss = 1.30 (153.3 examples/sec; 0.835 sec/batch)
2017-04-02 00:58:16.025112: step 2810, loss = 1.27 (154.5 examples/sec; 0.828 sec/batch)
2017-04-02 00:58:24.291412: step 2820, loss = 1.36 (154.8 examples/sec; 0.827 sec/batch)
2017-04-02 00:58:32.510651: step 2830, loss = 1.57 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 00:58:41.019828: step 2840, loss = 1.31 (150.4 examples/sec; 0.851 sec/batch)
2017-04-02 00:58:52.152300: step 2850, loss = 1.20 (115.0 examples/sec; 1.113 sec/batch)
2017-04-02 00:59:01.761286: step 2860, loss = 1.35 (133.2 examples/sec; 0.961 sec/batch)
2017-04-02 00:59:09.954727: step 2870, loss = 1.22 (156.2 examples/sec; 0.819 sec/batch)
2017-04-02 00:59:18.149911: step 2880, loss = 1.17 (156.2 examples/sec; 0.820 sec/batch)
2017-04-02 00:59:26.362889: step 2890, loss = 1.21 (155.9 examples/sec; 0.821 sec/batch)
INFO:tensorflow:global_step/sec: 1.09557
2017-04-02 00:59:39.019031: step 2900, loss = 1.26 (101.1 examples/sec; 1.266 sec/batch)
2017-04-02 00:59:47.225399: step 2910, loss = 1.25 (156.0 examples/sec; 0.821 sec/batch)
2017-04-02 00:59:56.147057: step 2920, loss = 1.45 (143.5 examples/sec; 0.892 sec/batch)
2017-04-02 01:00:05.770567: step 2930, loss = 1.23 (133.0 examples/sec; 0.962 sec/batch)
2017-04-02 01:00:13.960954: step 2940, loss = 1.07 (156.3 examples/sec; 0.819 sec/batch)
2017-04-02 01:00:22.203454: step 2950, loss = 1.27 (155.3 examples/sec; 0.824 sec/batch)
2017-04-02 01:00:30.438997: step 2960, loss = 1.42 (155.4 examples/sec; 0.824 sec/batch)
2017-04-02 01:00:40.785345: step 2970, loss = 1.10 (123.7 examples/sec; 1.035 sec/batch)
2017-04-02 01:00:49.001265: step 2980, loss = 1.16 (155.8 examples/sec; 0.822 sec/batch)
2017-04-02 01:01:01.678328: step 2990, loss = 1.25 (101.0 examples/sec; 1.268 sec/batch)
INFO:tensorflow:global_step/sec: 1.09889
2017-04-02 01:01:10.019999: step 3000, loss = 1.16 (153.4 examples/sec; 0.834 sec/batch)
2017-04-02 01:01:18.268202: step 3010, loss = 1.24 (155.2 examples/sec; 0.825 sec/batch)
2017-04-02 01:01:26.489543: step 3020, loss = 1.40 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 01:01:38.976216: step 3030, loss = 1.42 (102.5 examples/sec; 1.249 sec/batch)
2017-04-02 01:01:47.204774: step 3040, loss = 1.29 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 01:01:55.374985: step 3050, loss = 1.34 (156.7 examples/sec; 0.817 sec/batch)
2017-04-02 01:02:03.572740: step 3060, loss = 1.36 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 01:02:13.930239: step 3070, loss = 1.44 (123.6 examples/sec; 1.036 sec/batch)
2017-04-02 01:02:24.355303: step 3080, loss = 1.11 (122.8 examples/sec; 1.043 sec/batch)
2017-04-02 01:02:32.555441: step 3090, loss = 1.16 (156.1 examples/sec; 0.820 sec/batch)
INFO:tensorflow:global_step/sec: 1.07493
2017-04-02 01:02:43.049298: step 3100, loss = 1.35 (122.0 examples/sec; 1.049 sec/batch)
2017-04-02 01:02:53.461448: step 3110, loss = 1.31 (122.9 examples/sec; 1.041 sec/batch)
2017-04-02 01:03:01.648790: step 3120, loss = 1.53 (156.3 examples/sec; 0.819 sec/batch)
2017-04-02 01:03:09.850261: step 3130, loss = 1.17 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 01:03:20.181664: step 3140, loss = 1.28 (123.9 examples/sec; 1.033 sec/batch)
2017-04-02 01:03:30.224053: step 3150, loss = 1.31 (127.5 examples/sec; 1.004 sec/batch)
2017-04-02 01:03:41.265589: step 3160, loss = 1.17 (115.9 examples/sec; 1.104 sec/batch)
2017-04-02 01:03:49.475009: step 3170, loss = 1.22 (155.9 examples/sec; 0.821 sec/batch)
2017-04-02 01:03:59.802861: step 3180, loss = 1.13 (123.9 examples/sec; 1.033 sec/batch)
2017-04-02 01:04:07.992463: step 3190, loss = 1.43 (156.3 examples/sec; 0.819 sec/batch)
INFO:tensorflow:global_step/sec: 1.0236
2017-04-02 01:04:20.743090: step 3200, loss = 1.13 (100.4 examples/sec; 1.275 sec/batch)
2017-04-02 01:04:29.034779: step 3210, loss = 1.35 (154.4 examples/sec; 0.829 sec/batch)
2017-04-02 01:04:37.238866: step 3220, loss = 1.06 (156.0 examples/sec; 0.820 sec/batch)
2017-04-02 01:04:47.748694: step 3230, loss = 1.15 (121.8 examples/sec; 1.051 sec/batch)
2017-04-02 01:04:58.114183: step 3240, loss = 1.30 (123.5 examples/sec; 1.037 sec/batch)
2017-04-02 01:05:08.399566: step 3250, loss = 1.23 (124.4 examples/sec; 1.029 sec/batch)
2017-04-02 01:05:16.620079: step 3260, loss = 1.15 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 01:05:24.816709: step 3270, loss = 1.31 (156.2 examples/sec; 0.820 sec/batch)
2017-04-02 01:05:35.177669: step 3280, loss = 1.26 (123.5 examples/sec; 1.036 sec/batch)
2017-04-02 01:05:43.372716: step 3290, loss = 1.24 (156.2 examples/sec; 0.820 sec/batch)
INFO:tensorflow:global_step/sec: 1.09948
2017-04-02 01:05:51.694729: step 3300, loss = 1.37 (153.8 examples/sec; 0.832 sec/batch)
2017-04-02 01:06:02.097201: step 3310, loss = 1.22 (123.0 examples/sec; 1.040 sec/batch)
2017-04-02 01:06:12.435956: step 3320, loss = 1.01 (123.8 examples/sec; 1.034 sec/batch)
2017-04-02 01:06:20.671650: step 3330, loss = 1.17 (155.4 examples/sec; 0.824 sec/batch)
INFO:tensorflow:Saving checkpoints for 3333 into /tmp/cifar10_train/model.ckpt.
2017-04-02 01:06:31.148143: step 3340, loss = 1.05 (122.2 examples/sec; 1.048 sec/batch)
2017-04-02 01:06:39.371120: step 3350, loss = 1.05 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 01:06:48.024892: step 3360, loss = 1.15 (147.9 examples/sec; 0.865 sec/batch)
2017-04-02 01:07:00.077341: step 3370, loss = 1.13 (106.2 examples/sec; 1.205 sec/batch)
2017-04-02 01:07:08.296985: step 3380, loss = 1.27 (155.7 examples/sec; 0.822 sec/batch)
2017-04-02 01:07:16.483872: step 3390, loss = 1.28 (156.3 examples/sec; 0.819 sec/batch)
INFO:tensorflow:global_step/sec: 1.05305
2017-04-02 01:07:26.658768: step 3400, loss = 1.17 (125.8 examples/sec; 1.017 sec/batch)
2017-04-02 01:07:37.387538: step 3410, loss = 1.16 (119.3 examples/sec; 1.073 sec/batch)
2017-04-02 01:07:47.741862: step 3420, loss = 1.20 (123.6 examples/sec; 1.035 sec/batch)
2017-04-02 01:07:56.826277: step 3430, loss = 1.24 (140.9 examples/sec; 0.908 sec/batch)
2017-04-02 01:08:06.426690: step 3440, loss = 1.30 (133.3 examples/sec; 0.960 sec/batch)
2017-04-02 01:08:14.600910: step 3450, loss = 1.39 (156.6 examples/sec; 0.817 sec/batch)
2017-04-02 01:08:22.843853: step 3460, loss = 1.41 (155.3 examples/sec; 0.824 sec/batch)
2017-04-02 01:08:33.208182: step 3470, loss = 1.29 (123.5 examples/sec; 1.036 sec/batch)
2017-04-02 01:08:44.557902: step 3480, loss = 1.33 (112.8 examples/sec; 1.135 sec/batch)
2017-04-02 01:08:54.277646: step 3490, loss = 1.09 (131.7 examples/sec; 0.972 sec/batch)
INFO:tensorflow:global_step/sec: 1.01949
2017-04-02 01:09:04.745310: step 3500, loss = 1.19 (122.3 examples/sec; 1.047 sec/batch)
2017-04-02 01:09:12.959035: step 3510, loss = 1.09 (155.8 examples/sec; 0.821 sec/batch)
2017-04-02 01:09:23.205893: step 3520, loss = 1.19 (124.9 examples/sec; 1.025 sec/batch)
2017-04-02 01:09:33.657807: step 3530, loss = 1.11 (122.5 examples/sec; 1.045 sec/batch)
2017-04-02 01:09:44.027334: step 3540, loss = 1.25 (123.4 examples/sec; 1.037 sec/batch)
2017-04-02 01:09:52.192291: step 3550, loss = 0.99 (156.8 examples/sec; 0.816 sec/batch)
2017-04-02 01:10:00.393964: step 3560, loss = 1.11 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 01:10:09.889444: step 3570, loss = 1.25 (134.8 examples/sec; 0.950 sec/batch)
2017-04-02 01:10:21.078959: step 3580, loss = 1.16 (114.4 examples/sec; 1.119 sec/batch)
2017-04-02 01:10:29.346896: step 3590, loss = 1.22 (154.8 examples/sec; 0.827 sec/batch)
INFO:tensorflow:global_step/sec: 1.07609
2017-04-02 01:10:37.674258: step 3600, loss = 1.22 (153.7 examples/sec; 0.833 sec/batch)
2017-04-02 01:10:45.874797: step 3610, loss = 1.05 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 01:10:56.225091: step 3620, loss = 1.14 (123.7 examples/sec; 1.035 sec/batch)
2017-04-02 01:11:04.411638: step 3630, loss = 1.12 (156.4 examples/sec; 0.819 sec/batch)
2017-04-02 01:11:14.771072: step 3640, loss = 1.09 (123.6 examples/sec; 1.036 sec/batch)
2017-04-02 01:11:22.938478: step 3650, loss = 1.20 (156.7 examples/sec; 0.817 sec/batch)
2017-04-02 01:11:35.616444: step 3660, loss = 1.09 (101.0 examples/sec; 1.268 sec/batch)
2017-04-02 01:11:44.192718: step 3670, loss = 1.22 (149.2 examples/sec; 0.858 sec/batch)
2017-04-02 01:11:54.267738: step 3680, loss = 1.12 (127.0 examples/sec; 1.008 sec/batch)
2017-04-02 01:12:05.841864: step 3690, loss = 1.06 (110.6 examples/sec; 1.157 sec/batch)
INFO:tensorflow:global_step/sec: 1.02663
2017-04-02 01:12:15.080908: step 3700, loss = 0.86 (138.5 examples/sec; 0.924 sec/batch)
2017-04-02 01:12:23.371498: step 3710, loss = 0.95 (154.4 examples/sec; 0.829 sec/batch)
2017-04-02 01:12:35.889789: step 3720, loss = 1.29 (102.3 examples/sec; 1.252 sec/batch)
2017-04-02 01:12:44.081046: step 3730, loss = 0.97 (156.3 examples/sec; 0.819 sec/batch)
2017-04-02 01:12:56.729113: step 3740, loss = 1.16 (101.2 examples/sec; 1.265 sec/batch)
2017-04-02 01:13:04.913668: step 3750, loss = 1.16 (156.4 examples/sec; 0.818 sec/batch)
2017-04-02 01:13:15.439260: step 3760, loss = 1.12 (121.6 examples/sec; 1.053 sec/batch)
2017-04-02 01:13:25.759016: step 3770, loss = 1.22 (124.0 examples/sec; 1.032 sec/batch)
2017-04-02 01:13:36.434113: step 3780, loss = 1.16 (119.9 examples/sec; 1.068 sec/batch)
2017-04-02 01:13:44.606892: step 3790, loss = 0.99 (156.6 examples/sec; 0.817 sec/batch)
INFO:tensorflow:global_step/sec: 1.02198
2017-04-02 01:13:52.929822: step 3800, loss = 1.01 (153.8 examples/sec; 0.832 sec/batch)
2017-04-02 01:14:03.312340: step 3810, loss = 1.07 (123.3 examples/sec; 1.038 sec/batch)
2017-04-02 01:14:13.661530: step 3820, loss = 1.07 (123.7 examples/sec; 1.035 sec/batch)
2017-04-02 01:14:21.919820: step 3830, loss = 1.19 (155.0 examples/sec; 0.826 sec/batch)
2017-04-02 01:14:32.280193: step 3840, loss = 1.07 (123.5 examples/sec; 1.036 sec/batch)
2017-04-02 01:14:40.480254: step 3850, loss = 1.23 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 01:14:48.693832: step 3860, loss = 1.13 (155.8 examples/sec; 0.821 sec/batch)
2017-04-02 01:15:01.331102: step 3870, loss = 1.07 (101.3 examples/sec; 1.264 sec/batch)
2017-04-02 01:15:09.523697: step 3880, loss = 1.11 (156.2 examples/sec; 0.819 sec/batch)
2017-04-02 01:15:19.860191: step 3890, loss = 1.04 (123.8 examples/sec; 1.034 sec/batch)
INFO:tensorflow:global_step/sec: 1.04958
2017-04-02 01:15:28.206122: step 3900, loss = 1.15 (153.4 examples/sec; 0.835 sec/batch)
2017-04-02 01:15:38.573344: step 3910, loss = 1.05 (123.5 examples/sec; 1.037 sec/batch)
2017-04-02 01:15:48.942548: step 3920, loss = 1.18 (123.4 examples/sec; 1.037 sec/batch)
2017-04-02 01:15:57.150513: step 3930, loss = 1.38 (155.9 examples/sec; 0.821 sec/batch)
2017-04-02 01:16:07.132449: step 3940, loss = 1.09 (128.2 examples/sec; 0.998 sec/batch)
2017-04-02 01:16:15.717549: step 3950, loss = 1.21 (149.1 examples/sec; 0.859 sec/batch)
INFO:tensorflow:Saving checkpoints for 3960 into /tmp/cifar10_train/model.ckpt.
2017-04-02 01:16:24.158930: step 3960, loss = 1.19 (151.6 examples/sec; 0.844 sec/batch)
2017-04-02 01:16:32.962712: step 3970, loss = 0.79 (145.4 examples/sec; 0.880 sec/batch)
2017-04-02 01:16:44.863398: step 3980, loss = 1.04 (107.6 examples/sec; 1.190 sec/batch)
2017-04-02 01:16:55.217613: step 3990, loss = 1.28 (123.6 examples/sec; 1.035 sec/batch)
INFO:tensorflow:global_step/sec: 1.04888
2017-04-02 01:17:03.545665: step 4000, loss = 1.10 (153.7 examples/sec; 0.833 sec/batch)
2017-04-02 01:17:13.916828: step 4010, loss = 1.13 (123.4 examples/sec; 1.037 sec/batch)
2017-04-02 01:17:22.132042: step 4020, loss = 1.01 (155.8 examples/sec; 0.822 sec/batch)
2017-04-02 01:17:34.751956: step 4030, loss = 1.17 (101.4 examples/sec; 1.262 sec/batch)
2017-04-02 01:17:42.978378: step 4040, loss = 1.07 (155.6 examples/sec; 0.823 sec/batch)
2017-04-02 01:17:51.202079: step 4050, loss = 0.87 (155.6 examples/sec; 0.822 sec/batch)
2017-04-02 01:18:01.527383: step 4060, loss = 1.15 (124.0 examples/sec; 1.033 sec/batch)
2017-04-02 01:18:11.862400: step 4070, loss = 1.04 (123.9 examples/sec; 1.034 sec/batch)
2017-04-02 01:18:20.067897: step 4080, loss = 1.19 (156.0 examples/sec; 0.821 sec/batch)
2017-04-02 01:18:28.334581: step 4090, loss = 1.26 (154.8 examples/sec; 0.827 sec/batch)
INFO:tensorflow:global_step/sec: 1.05698
2017-04-02 01:18:38.157201: step 4100, loss = 1.18 (130.3 examples/sec; 0.982 sec/batch)
2017-04-02 01:18:47.342558: step 4110, loss = 1.14 (139.4 examples/sec; 0.919 sec/batch)
2017-04-02 01:18:55.585535: step 4120, loss = 1.23 (155.3 examples/sec; 0.824 sec/batch)
2017-04-02 01:19:05.890299: step 4130, loss = 1.04 (124.2 examples/sec; 1.030 sec/batch)
2017-04-02 01:19:16.874655: step 4140, loss = 1.03 (116.5 examples/sec; 1.098 sec/batch)
2017-04-02 01:19:26.672871: step 4150, loss = 1.04 (130.6 examples/sec; 0.980 sec/batch)
2017-04-02 01:19:34.872641: step 4160, loss = 1.13 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 01:19:43.066020: step 4170, loss = 0.94 (156.2 examples/sec; 0.819 sec/batch)
2017-04-02 01:19:51.251749: step 4180, loss = 1.17 (156.4 examples/sec; 0.819 sec/batch)
2017-04-02 01:20:01.607851: step 4190, loss = 1.03 (123.6 examples/sec; 1.036 sec/batch)
INFO:tensorflow:global_step/sec: 1.08933
2017-04-02 01:20:09.954394: step 4200, loss = 1.03 (153.4 examples/sec; 0.835 sec/batch)
2017-04-02 01:20:18.206125: step 4210, loss = 1.02 (155.1 examples/sec; 0.825 sec/batch)
2017-04-02 01:20:29.128701: step 4220, loss = 1.12 (117.2 examples/sec; 1.092 sec/batch)
2017-04-02 01:20:38.886211: step 4230, loss = 1.04 (131.2 examples/sec; 0.976 sec/batch)
2017-04-02 01:20:49.398959: step 4240, loss = 1.08 (121.8 examples/sec; 1.051 sec/batch)
2017-04-02 01:20:59.319529: step 4250, loss = 1.07 (129.0 examples/sec; 0.992 sec/batch)
2017-04-02 01:21:07.930980: step 4260, loss = 1.05 (148.6 examples/sec; 0.861 sec/batch)
2017-04-02 01:21:16.127981: step 4270, loss = 1.09 (156.2 examples/sec; 0.820 sec/batch)
2017-04-02 01:21:24.592456: step 4280, loss = 1.16 (151.2 examples/sec; 0.846 sec/batch)
2017-04-02 01:21:34.677108: step 4290, loss = 1.06 (126.9 examples/sec; 1.008 sec/batch)
INFO:tensorflow:global_step/sec: 1.03439
2017-04-02 01:21:46.631801: step 4300, loss = 1.10 (107.1 examples/sec; 1.195 sec/batch)
2017-04-02 01:21:55.687455: step 4310, loss = 1.10 (141.3 examples/sec; 0.906 sec/batch)
2017-04-02 01:22:08.294045: step 4320, loss = 1.05 (101.5 examples/sec; 1.261 sec/batch)
2017-04-02 01:22:16.506674: step 4330, loss = 1.05 (155.9 examples/sec; 0.821 sec/batch)
2017-04-02 01:22:25.501291: step 4340, loss = 1.19 (142.3 examples/sec; 0.899 sec/batch)
2017-04-02 01:22:37.267708: step 4350, loss = 1.16 (108.8 examples/sec; 1.177 sec/batch)
2017-04-02 01:22:45.439584: step 4360, loss = 1.20 (156.6 examples/sec; 0.817 sec/batch)
2017-04-02 01:22:55.313146: step 4370, loss = 1.11 (129.6 examples/sec; 0.987 sec/batch)
2017-04-02 01:23:03.959971: step 4380, loss = 1.24 (148.0 examples/sec; 0.865 sec/batch)
2017-04-02 01:23:12.170575: step 4390, loss = 1.01 (155.9 examples/sec; 0.821 sec/batch)
INFO:tensorflow:global_step/sec: 1.06228
2017-04-02 01:23:20.769108: step 4400, loss = 0.96 (148.9 examples/sec; 0.860 sec/batch)
2017-04-02 01:23:30.782807: step 4410, loss = 1.09 (127.8 examples/sec; 1.001 sec/batch)
2017-04-02 01:23:43.114817: step 4420, loss = 1.13 (103.8 examples/sec; 1.233 sec/batch)
2017-04-02 01:23:51.932841: step 4430, loss = 1.13 (145.2 examples/sec; 0.882 sec/batch)
2017-04-02 01:24:02.314633: step 4440, loss = 0.98 (123.3 examples/sec; 1.038 sec/batch)
2017-04-02 01:24:12.679705: step 4450, loss = 0.97 (123.5 examples/sec; 1.037 sec/batch)
2017-04-02 01:24:20.891451: step 4460, loss = 1.12 (155.9 examples/sec; 0.821 sec/batch)
2017-04-02 01:24:29.167410: step 4470, loss = 1.15 (154.7 examples/sec; 0.828 sec/batch)
2017-04-02 01:24:37.342718: step 4480, loss = 1.31 (156.6 examples/sec; 0.818 sec/batch)
2017-04-02 01:24:45.540254: step 4490, loss = 1.06 (156.1 examples/sec; 0.820 sec/batch)
INFO:tensorflow:global_step/sec: 1.05409
2017-04-02 01:24:55.638213: step 4500, loss = 1.08 (126.8 examples/sec; 1.010 sec/batch)
2017-04-02 01:25:06.385392: step 4510, loss = 1.06 (119.1 examples/sec; 1.075 sec/batch)
2017-04-02 01:25:14.621199: step 4520, loss = 1.02 (155.4 examples/sec; 0.824 sec/batch)
2017-04-02 01:25:22.810930: step 4530, loss = 1.09 (156.3 examples/sec; 0.819 sec/batch)
2017-04-02 01:25:31.012469: step 4540, loss = 1.03 (156.1 examples/sec; 0.820 sec/batch)
2017-04-02 01:25:41.343136: step 4550, loss = 1.09 (123.9 examples/sec; 1.033 sec/batch)
2017-04-02 01:25:49.533978: step 4560, loss = 1.24 (156.3 examples/sec; 0.819 sec/batch)
2017-04-02 01:25:57.725825: step 4570, loss = 0.99 (156.3 examples/sec; 0.819 sec/batch)
2017-04-02 01:26:07.734320: step 4580, loss = 1.11 (127.9 examples/sec; 1.001 sec/batch)
2017-04-02 01:26:18.456187: step 4590, loss = 1.22 (119.4 examples/sec; 1.072 sec/batch)
INFO:tensorflow:Saving checkpoints for 4597 into /tmp/cifar10_train/model.ckpt.
INFO:tensorflow:global_step/sec: 1.09484
2017-04-02 01:26:26.972980: step 4600, loss = 0.93 (150.3 examples/sec; 0.852 sec/batch)
2017-04-02 01:26:37.346454: step 4610, loss = 0.91 (123.4 examples/sec; 1.037 sec/batch)
2017-04-02 01:26:47.710495: step 4620, loss = 1.20 (123.5 examples/sec; 1.036 sec/batch)
2017-04-02 01:26:55.905469: step 4630, loss = 1.23 (156.2 examples/sec; 0.819 sec/batch)
2017-04-02 01:27:04.101146: step 4640, loss = 1.19 (156.2 examples/sec; 0.820 sec/batch)
2017-04-02 01:27:16.740272: step 4650, loss = 1.06 (101.3 examples/sec; 1.264 sec/batch)
2017-04-02 01:27:24.912176: step 4660, loss = 1.29 (156.6 examples/sec; 0.817 sec/batch)
2017-04-02 01:27:33.081739: step 4670, loss = 1.15 (156.7 examples/sec; 0.817 sec/batch)
2017-04-02 01:27:43.454315: step 4680, loss = 1.07 (123.4 examples/sec; 1.037 sec/batch)
2017-04-02 01:27:55.306853: step 4690, loss = 0.94 (108.0 examples/sec; 1.185 sec/batch)
INFO:tensorflow:global_step/sec: 1.02817
2017-04-02 01:28:04.233219: step 4700, loss = 0.99 (143.4 examples/sec; 0.893 sec/batch)
2017-04-02 01:28:14.792433: step 4710, loss = 0.94 (121.2 examples/sec; 1.056 sec/batch)
2017-04-02 01:28:23.048374: step 4720, loss = 1.08 (155.0 examples/sec; 0.826 sec/batch)
2017-04-02 01:28:31.279019: step 4730, loss = 1.10 (155.5 examples/sec; 0.823 sec/batch)
2017-04-02 01:28:39.762967: step 4740, loss = 1.17 (150.9 examples/sec; 0.848 sec/batch)
2017-04-02 01:28:47.978921: step 4750, loss = 1.02 (155.8 examples/sec; 0.822 sec/batch)
2017-04-02 01:28:58.333441: step 4760, loss = 1.12 (123.6 examples/sec; 1.035 sec/batch)
2017-04-02 01:29:08.664770: step 4770, loss = 1.03 (123.9 examples/sec; 1.033 sec/batch)
2017-04-02 01:29:16.829979: step 4780, loss = 0.96 (156.8 examples/sec; 0.817 sec/batch)
2017-04-02 01:29:27.198749: step 4790, loss = 0.92 (123.4 examples/sec; 1.037 sec/batch)
INFO:tensorflow:global_step/sec: 1.09556
2017-04-02 01:29:35.510692: step 4800, loss = 1.04 (154.0 examples/sec; 0.831 sec/batch)
2017-04-02 01:29:46.817887: step 4810, loss = 1.00 (113.2 examples/sec; 1.131 sec/batch)
2017-04-02 01:29:55.023013: step 4820, loss = 1.04 (156.0 examples/sec; 0.821 sec/batch)
2017-04-02 01:30:03.192565: step 4830, loss = 1.16 (156.7 examples/sec; 0.817 sec/batch)
2017-04-02 01:30:13.545439: step 4840, loss = 1.07 (123.6 examples/sec; 1.035 sec/batch)
2017-04-02 01:30:21.760412: step 4850, loss = 0.96 (155.8 examples/sec; 0.821 sec/batch)
2017-04-02 01:30:34.442320: step 4860, loss = 1.03 (100.9 examples/sec; 1.268 sec/batch)
2017-04-02 01:30:42.600433: step 4870, loss = 1.15 (156.9 examples/sec; 0.816 sec/batch)
2017-04-02 01:30:50.746673: step 4880, loss = 0.91 (157.1 examples/sec; 0.815 sec/batch)
2017-04-02 01:31:01.092740: step 4890, loss = 0.94 (123.7 examples/sec; 1.035 sec/batch)
INFO:tensorflow:global_step/sec: 1.05426
2017-04-02 01:31:10.366527: step 4900, loss = 0.79 (138.0 examples/sec; 0.927 sec/batch)
2017-04-02 01:31:19.908751: step 4910, loss = 1.03 (134.1 examples/sec; 0.954 sec/batch)
2017-04-02 01:31:28.089185: step 4920, loss = 0.98 (156.5 examples/sec; 0.818 sec/batch)
2017-04-02 01:31:38.443663: step 4930, loss = 0.92 (123.6 examples/sec; 1.035 sec/batch)
2017-04-02 01:31:49.148475: step 4940, loss = 0.98 (119.6 examples/sec; 1.070 sec/batch)
2017-04-02 01:31:59.220002: step 4950, loss = 1.13 (127.1 examples/sec; 1.007 sec/batch)
2017-04-02 01:32:07.379355: step 4960, loss = 1.18 (156.9 examples/sec; 0.816 sec/batch)
2017-04-02 01:32:15.539825: step 4970, loss = 0.97 (156.9 examples/sec; 0.816 sec/batch)
2017-04-02 01:32:23.786316: step 4980, loss = 1.04 (155.2 examples/sec; 0.825 sec/batch)
2017-04-02 01:32:34.108083: step 4990, loss = 1.14 (124.0 examples/sec; 1.032 sec/batch)
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/cifar10_train/model.ckpt.
'''